In [0]:
wrkbase = '/content/'



Wr = 0.299
Wg = 0.587
Wb = 0.114
Umax = 0.436
Vmax = 0.615

scaleYp = 256.0/220.0
scaleUVp = 256.0/225.0


s960 = (1440,960)
s2880 = (3968,2880)


!7z e '/content/drive/My Drive/TFMLz/ESRGAN_oldarch/models/falcoon300.7z' -o/content
!sudo apt-get -y install libffms2-4 
!pip install ffms2



%cd /content/drive/My Drive/TFMLz/ESRGAN_oldarch
import os
import cv2
import numpy as np
import shutil
import torch
import torchvision.transforms as TF
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import architecture as arch
import modelz
import datas
import configs
from utils.config import Config
import ffms2
from PIL import Image 

try:
  os.mkdir(wrkbase+'oY')
  os.mkdir(wrkbase+'oU')
  os.mkdir(wrkbase+'oV')
  os.mkdir(wrkbase+'wY')
  os.mkdir(wrkbase+'wU')
  os.mkdir(wrkbase+'wV')
except OSError:
    pass


def mkESRGAN(model_path,scale):

  if not os.path.isfile(model_path):
    model_path = 'models/'+model_path+'.pth'

  
  model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=scale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
  model.load_state_dict(torch.load(model_path), strict=True)
  model.eval()
  for k, v in model.named_parameters():
      v.requires_grad = False
  return model

def RshapeFrame(xfr):
  bY = np.array(xfr.planes[0]).reshape((480,768))[:,:720]
  bU = np.array(xfr.planes[1]).reshape((480,384))[:240,:360]
  bV = np.array(xfr.planes[2]).reshape((480,384))[:240,:360]
  return bY,bU,bV

def interlace(imgL, imgR, h, w):
    inter = np.empty((h, w), imgL.dtype)
    inter[:h:2, :w] = imgR[:h:2, :w]
    inter[1:h:2, :w] = imgL[1:h:2, :w]
    return inter

def mkWRK(title='V004_b',workf='wrk.m2v'):
  wff = wrkbase+workf
  if not os.path.isfile(wff):
    %cd /content
    !git clone https://github.com/IbrahimKurz/{title}.git
    !chmod 755 /content/{title}/README.sh
    !/content/{title}/README.sh
    shutil.rmtree(wrkbase+title, ignore_errors=True)
  vsource = ffms2.VideoSource(wff)
  return vsource

def PreVwrk(de,skip=0,Len=2):
  idx=skip
  Len*=5
  Len+=idx
  while idx < Len:
    bY, bU, bV = RshapeFrame(de.get_frame(idx))
    print('%d=' % idx,end='')
    display(Image.fromarray(bY))
    idx+=1



  

def DeInterLace(de,inta,skip=0,Len=-1,StartPak=0):
  idx=skip
  if StartPak !=0:
    idx+=StartPak*5

  fcot = de.properties.NumFrames
  if Len!=-1:
    fcot = idx+Len

  while idx < fcot:  
    bY, bU, bV = RshapeFrame(de.get_frame(idx))
    if idx%5 == inta:
      idx+=1
      nxbY, nxbU, nxbV = RshapeFrame(de.get_frame(idx))
      bY = interlace(bY,nxbY,480,720)
      bU = interlace(bU,nxbU,240,360)
      bV = interlace(bV,nxbV,240,360)
  
  

    strfmt = '%so%s/x%05d.png' % (wrkbase,'%s',idx)
    cv2.imwrite(strfmt % 'Y',bY)
    cv2.imwrite(strfmt % 'U',bU)
    cv2.imwrite(strfmt % 'V',bV)
    idx+=1


def toRGB(fiY,fiU,fiV,slif=None,scaleYnUV=(True,True)):

  isf = isinstance(fiY, str)

  if fiU is None:
    tYpl = fiY[0]
    tUpl = fiY[1]
    tVpl = fiY[2]
  else:
    if isf:
      tYpl = cv2.imread(fiY, cv2.IMREAD_GRAYSCALE).astype(np.float32)
      tUpl = cv2.imread(fiU, cv2.IMREAD_GRAYSCALE).astype(np.float32)
      tVpl = cv2.imread(fiV, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    else:
      tYpl = fiY
      tUpl = fiU
      tVpl = fiV
    if tYpl.shape != tUpl.shape:
      tUpl = cv2.resize(tUpl,None,fx=2,fy=2, interpolation=cv2.INTER_LINEAR)
      tVpl = cv2.resize(tVpl,None,fx=2,fy=2, interpolation=cv2.INTER_LINEAR)
  

  if scaleYnUV[0]:
    tYpl-=16
    tYpl*=scaleYp

  if scaleYnUV[1]:
    tUpl-=16
    tUpl*=scaleUVp
    tVpl-=16
    tVpl*=scaleUVp

  if isf:
    tUpl -= 128.0
    tVpl -= 128.0

  if slif is not None:
    tYpl+=slif[0]
    tUpl+=slif[1]
    tVpl+=slif[2]



  tfR = tYpl + tVpl*(1-Wr)/Vmax
  tfG = tYpl - (tUpl*Wb*(1-Wb)/Umax+tVpl*Wr*(1-Wr)/Vmax)/Wg
  tfB = tYpl + tUpl*(1-Wb)/Umax
  return np.asarray([tfR,tfG,tfB])

def toYUV(inRGB):
  fY = Wr*inRGB[0]+Wg*inRGB[1]+Wb*inRGB[2]
  fU = Umax*(inRGB[2]-fY)/(1-Wb)
  fV = Vmax*(inRGB[0]-fY)/(1-Wr)
  fU+=0.5
  fV+=0.5
  return np.asarray([fY,fU,fV])

def PreVde(start):
  start*=4
  wYr = wrkbase+'oYr'
  wY = wrkbase+'oY'
  if not os.path.isdir(wYr):
    os.rename(wY, wYr)

  flist = os.listdir(wYr)
  flist.sort()
  flist = flist[start:start+4]
  wYr+='/'

  for f in flist:
    display(Image.open(wYr+f))

def Yrange(start,Len):
  start*=4
  wYr = wrkbase+'oYr'
  wY = wrkbase+'oY'
  if not os.path.isdir(wYr):
    os.rename(wY, wYr)
  
  flist = os.listdir(wYr)
  flist.sort()
  flist = flist[start:start+Len*24]
  
  shutil.rmtree(wY, ignore_errors=True)
  os.mkdir(wY)

  wYr+='/'
  wY+='/'
  for f in flist:
    os.link(wYr+f,wY+f)

def dnSS(img,scca):
  
  return np.asarray([cv2.resize(img[0],scca, interpolation=cv2.INTER_AREA),cv2.resize(img[1],scca, interpolation=cv2.INTER_AREA),cv2.resize(img[2],scca, interpolation=cv2.INTER_AREA)])

def to2880():
  wY = wrkbase+'wY'
  flist = os.listdir(wY)
  flist.sort()

  #flist=flist[7:10]
  
  for f in flist:
    wrkY = wrkbase+'wY/'+f
    wrkU = wrkbase+'wU/'+f
    wrkV = wrkbase+'wV/'+f
    img = torch.from_numpy(toRGB(wrkY,wrkU,wrkV)/255.0)
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)
    ot1 = model_1st(img_LR).data.squeeze().cpu().numpy()
    if ot1.shape[1]!=960:
      ot1=dnSS(ot1,s960)


    '''
    ycole = [[
              ot1[:,:512,:512],
              ot1[:,:512,464:976],
              ot1[:,:512,928:]],
             [
              ot1[:,448:,:512],
              ot1[:,448:,464:976],
              ot1[:,448:,928:]]]
    
    for y in range(2):
      for x in range(3):
        img = torch.from_numpy(ycole[y][x])
        img_LR = img.unsqueeze(0)
        img_LR = img_LR.to(device)
        ycole[y][x] = model_2nd(img_LR).data.squeeze().cpu().numpy()

    for y in range(2):
      ycole[y][0] = ycole[y][0][:,:,:-96] 
      ycole[y][1] = ycole[y][1][:,:,96:-96] 
      ycole[y][2] = ycole[y][2][:,:,96:]
      ycole[y] = np.concatenate(ycole[y],axis=2)

    ycole[0]=ycole[0][:,:-128,:]
    ycole[1]=ycole[1][:,128:,:]

    ycole = np.concatenate(ycole,axis=1)

    ycole = dnSS(ycole,s2880)
    '''
    #display(Image.fromarray(np.transpose(ycole*255.0,(1, 2, 0)).astype(np.uint8)))


###
device = torch.device('cuda')
model_1st =  mkESRGAN('FatalimiX',4).to(device)
model_2nd =  mkESRGAN('/content/falcoon300.pth',4).to(device)

  

rg1 = range(1)
rg2 = range(2)

config = Config.from_file('configs/config_test.py')

# preparing datasets
normalize1 = TF.Normalize(config.mean, [1.0, 1.0, 1.0])
normalize2 = TF.Normalize([0, 0, 0], config.std)
trans = TF.Compose([TF.ToTensor(), normalize1, normalize2, ])

revmean = [-x for x in config.mean]
revstd = [1.0 / x for x in config.std]
revnormalize1 = TF.Normalize([0.0, 0.0, 0.0], revstd)
revnormalize2 = TF.Normalize(revmean, [1.0, 1.0, 1.0])
revNormalize = TF.Compose([revnormalize1, revnormalize2])

revtrans = TF.Compose([revnormalize1, revnormalize2]) #TF.ToPILImage()

testset = datas.MySequence(config.testset_root, trans, config.test_size, config.test_crop_size, config.inter_frames)
sampler = torch.utils.data.SequentialSampler(testset)
validationloader = torch.utils.data.DataLoader(testset, sampler=sampler, batch_size=1, shuffle=False, num_workers=1)


# model
model = getattr(modelz, config.model)(config.pwc_path).cuda()
model = nn.DataParallel(model)
tot_frames = 0
dict1 = torch.load(config.checkpoint)
model.load_state_dict(dict1['model_state_dict'])

###

def dumpYUV(output,patha):
  ff = toYUV(output)*255.0

  ff0 = (ff[0]/scaleYp)+16
  ff1 = (ff[1]/scaleUVp)+16
  ff2 = (ff[2]/scaleUVp)+16

  cv2.imwrite('/content/wY/'+patha, ff0)
  cv2.imwrite('/content/wU/'+patha, ff1)
  cv2.imwrite('/content/wV/'+patha, ff2)

def SloMo():
    global tot_frames
    retImg = []
   
    

    vrgg = rg1
    vrgk = 1
    

    with torch.no_grad():
        for validationIndex, validationData in enumerate(validationloader, 0):
            sample, psyh = validationData

            naubase = psyh[0][0]
            frame0 = sample[0]
            frame1 = sample[1]
            frame2 = sample[2]
            frame3 = sample[3]

            #print(str(frame0.dtype)+',lay='+str(frame0.layout))
            #print(frame0)

            I0 = frame0.cuda()
            I3 = frame3.cuda()

            I1 = frame1.cuda()
            I2 = frame2.cuda()

            
            print(naubase)
            revtrans(I0.clone().cpu()[0]) #.save(svba)
            #dumpYUV(vymg,naubase)
            os.link('/content/oY/'+naubase,'/content/wY/'+naubase)
            os.link('/content/oU/'+naubase,'/content/wU/'+naubase)
            os.link('/content/oV/'+naubase,'/content/wV/'+naubase)

            
                
            

            
            for tt in vrgg:
                x = config.inter_frames
                t = 1.0/(x+1) * (tt + 1)
                #print(t)


                # record duration time
                

                output = model(I0, I1, I2, I3, t)
                It_warp = output
                
                
                tot_frames += 1
                

                
                vymg2 = revtrans(It_warp.cpu()[0]).numpy()
                dumpYUV(vymg2,psyh[1][0]+ '_' + str(tt) + '.png')




            
            if vrgk==1:
              vrgk = 2
              vrgg = rg2
            elif vrgk == 2:
              vrgk = 1
              vrgg = rg1
               
               
                    




   
    




In [0]:
wrk = mkWRK(title='V004_b',workf='wrk.m2v')
PreVwrk(wrk,skip=9)

In [0]:
DeInterLace(wrk,3)
#!ffmpeg -v quiet -framerate 1 -i {wrkbase}wY/x%*.png -c:v libx264 -crf 30 -pix_fmt yuv420p {wrkbase}tetview.264

In [0]:
#PreVde(2800)
Yrange(2800,20)

In [0]:
SloMo()

In [0]:
to2880()